In [43]:
!pip install requests
!pip install lxml
!pip install beautifulsoup4

In [1]:
import requests
from bs4 import BeautifulSoup
import re

```
目标爬取百度学术的论文标题、引用数、发表年份
先分解任务，从单页内容开始
```

In [27]:
url='https://xueshu.baidu.com/s?wd=resnet&rsv_bp=0&tn=SE_baiduxueshu_c1gjeupa&rsv_spt=3&ie=utf-8&f=8&rsv_sug2=0&sc_f_para=sc_tasktype%3D%7BfirstSimpleSearch%7D'
strhtml=requests.get(url)
soup=BeautifulSoup(strhtml.text, 'lxml')

In [8]:
def get_title(soup, sel):
    title = soup.select(sel)
    if len(title) > 0:
        title_d = title[0].get_text()
        return title_d
    else:
        return None

def get_count(soup, sel):
    count = soup.select(sel)
    # 排除下面2种情况时，引用数为0
    if len(count) == 0:
        return 0
    count_d = count[0].get_text()
    count_d = re.findall(r'\d+', count_d)
    if len(count_d) > 0:
        return count_d[0]
    sel = sel.replace('span:nth-child(2)', 'span:nth-child(3)')
    count = soup.select(sel)
    count_d = count[0].get_text()
    count_d = re.findall(r'\d+', count_d)
    return count_d[0]
    

def get_year(soup, sel):
    year = soup.select(sel)
    # 存在没有年份的情况
    if len(year) == 0:
        return -1
    year_d = year[0].get('data-year')
    return year_d

In [47]:
tmp = [str(i)+' ' for i in range(31, 40)]
tmp.append('31 0')
for i in tmp:
    sel0 = '#\\{} > div.sc_content > h3 > a'.format(i)
    title = get_title(sel0)
    print(title)
    sel1 = '#\\{} > div.sc_content > div.sc_info > span:nth-child(2) > a'.format(i)
    count = get_count(sel1)
    print(count)
    sel2 = '#\\{} > div.sc_content > div.sc_info > span.sc_time'.format(i)
    year = get_year(sel2)
    print(year)

Inception-v4, Inception-ResNet and the Impact of Residual Connections on Learning
1495
2016
Wider or Deeper: Revisiting the ResNet Model for Visual Recognition
117
2016
Brazilian Network for HIV Drug Resistance Surveillance (HIV-BResNet): a new survey of HIV recent diagnosed individuals collected in ...
119
2009
VoxResNet: Deep voxelwise residual networks for brain segmentation from 3D MR images - ScienceDirect
64
2018
Kinetic of the NO removal by nonthermal plasma in N2/NO/C2H4 mixtures
75
2000
In Vitro Transcription of the Viral-Specific Sequences Presnet in the Chromatin of Cells Transformed by Simian Virus 40
43
1973
Secondary Mediation and Regression Analyses of the PTClinResNet Database: Determining Causal Relationships Among the International C...
22
2011
Linking trait variation to the environment
13
2017
Act to staunch loss of research data
13
2015
Study of Ammonia Formation During the Purge of a Lean NO
11
2009


```
上个单元，告一段落，可以适当休息一下
接下来爬取多页结果
```

In [3]:
def trans(n):
    s = '3' + str(n)
    if len(s) > 2:
        s = s[:2] + ' ' + s[2:]
    return s

In [33]:
titles = []
counts = []
years = []
flag = True
i = 0
while flag:
    url = 'https://xueshu.baidu.com/s?wd=resnet&pn={}&tn=SE_baiduxueshu_c1gjeupa&ie \
        =utf-8&sc_f_para=sc_tasktype%3D%7BfirstSimpleSearch%7D&sc_hit=1'.format(i)
    strhtml=requests.get(url)
    soup=BeautifulSoup(strhtml.text, 'lxml')
    for j in range(1, 11):
        tar = trans(i + j)
        sel0 = '#\\{} > div.sc_content > h3 > a'.format(tar)
        title = get_title(soup, sel0)
        if title == None:
            flag = False
            break
        titles.append(title)
        sel1 = '#\\{} > div.sc_content > div.sc_info > span:nth-child(2) > a'.format(tar)
        count = get_count(soup, sel1)
        counts.append(count)
        sel2 = '#\\{} > div.sc_content > div.sc_info > span.sc_time'.format(tar)
        year = get_year(soup, sel2)
        years.append(year)
    i += 10

In [34]:
len(titles)

422

In [30]:
titles = []
counts = []
years = []
ckpt = 0
def retry_scrape(ckpt):
    # 增加重试功能，应对网络不稳定情况
    i = ckpt // 10
    last_j = ckpt % 10 + 1
    while 1:
        url = 'https://xueshu.baidu.com/s?wd=resnet&pn={}&tn=SE_baiduxueshu_c1gjeupa&ie \
            =utf-8&sc_f_para=sc_tasktype%3D%7BfirstSimpleSearch%7D&sc_hit=1'.format(i)
        strhtml=requests.get(url)
        soup=BeautifulSoup(strhtml.text, 'lxml')
        for j in range(last_j, 11):
            tar = trans(i + j)
            sel0 = '#\\{} > div.sc_content > h3 > a'.format(tar)
            title = get_title(soup, sel0)
            if title == None:
                return i + j
            titles.append(title)
            sel1 = '#\\{} > div.sc_content > div.sc_info > span:nth-child(2) > a'.format(tar)
            count = get_count(soup, sel1)
            counts.append(count)
            sel2 = '#\\{} > div.sc_content > div.sc_info > span.sc_time'.format(tar)
            year = get_year(soup, sel2)
            years.append(year)
        last_j = 1
        i += 10

In [31]:
ckpt = retry_scrape(ckpt)

```
将结果保存为excel文件
```

In [24]:
!pip install pandas
!pip install XlsxWriter

In [20]:
import pandas as pd 

In [21]:
data = {  
    '文章': titles,  
    '引用数': counts,
    '发表时间': years
}  

In [26]:
# 将字典转换为DataFrame  
df = pd.DataFrame(data)  
  
# 创建一个ExcelWriter对象，你可以选择一个文件名或者一个已经打开的文件对象  
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')  
  
# 将DataFrame写入到Excel文件的一个sheet中  
df.to_excel(writer, sheet_name='Sheet1', index=False)  

# 关闭ExcelWriter对象，这时会触发将数据写入到Excel文件中  
writer.close()